#### DataSet物件

In [ ]:
class StockDataset(Dataset):
    def __init__(self, x_text, x_price, y=None, window=5):
        self.window  = window
        self.x_text  = x_text
        self.x_price = x_price
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        if len(x_text) != len(x_price):
            raise Exception("Sorry, len(x_text) != len(x_price)")
        
    def __len__(self):
        return len(self.x_text) - self.window
    
    def __getitem__(self, index):
        x_text  = self.x_text[index:index+self.window]
        x_price = self.x_price[index:index+self.window]
        if self.y is not None:
            Y = self.y[index]
            return x_text, x_price, Y
        else:
            return x_text, x_price

#### 用DataSet建立DataLoader

In [ ]:
batch_size   = 128
train_set    = StockDataset(train_price, train_text, train_label)
val_set      = StockDataset(valid_price, valid_text, valid_label)
test_set     = StockDataset(test_price, test_text, test_label)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_set, batch_size=batch_size, shuffle=True)

#### 訓練時如何使用DataLoader

In [ ]:
for i, data in enumerate(train_loader):
    
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(data[0].cuda()) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數 # data.cuda()將記憶體中的數據放到GPU的顯存上，因為batch = 128，所以有128個預測結果，shape=(128,共有幾類)
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy()) # np.argmax(train_pred.cpu().data.numpy(), axis=1)會選出每次預測中機率最高的，然後跟data[1]對比，看有幾個正確
        train_loss += batch_loss.item() #.item()可以取得loss值